# Team 2: Evaluating Urban Trees in Los Angeles

This tool uses imagery, census block boundaries, and tree canopy metadata (in the form of a csv) in Los Angeles to identify census blocks in most need of increased tree canopy. It evaluates census blocks' existing tree canopy in 2016 (percent of census block area with tree canopy), percent of area identified in having potential for additional tree canopy as of 2016, and an updated percent of existing tree canopy in 2022.

We used an area in the San Fernando Valley in Los Angeles for this proof of concept.

GEOG 6293 | Eden Rotonda and Maya O'Brien

In [ ]:
# Import necessary modules

import geowombat as gw
import geopandas as gpd
import pandas as pd
import rasterio
from rasterio.warp import reproject, Resampling, calculate_default_transform
from rasterio.mask import mask
from rasterio.coords import BoundingBox
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from geowombat.ml import  fit_predict

In [ ]:
# Create variables for census block shapefile and tree canopy metric table from Tree People

study_area = gpd.read_file("./data/StudyArea_CensusBlock.shp")
tree_canopy_metrics = gpd.read_file("./data/TreeCanopy_Metric2010.csv")

In [ ]:
# Convert necessary fields to int, drop unecessary fields, and merge tree canopy info to shapefile

tree_canopy_metrics['TC_ID'] = tree_canopy_metrics['TC_ID'].astype('int64')

tree_canopy_metrics = tree_canopy_metrics.drop(labels=['OID_', 'TC_E_A', 'TC_Pv_A', 'TC_Land_A', 'TC_Pi_A',
       'TC_P_A', 'TC_Pv_P', 'TC_Pi_P'], axis=1)

studyarea_lc = pd.merge(study_area, tree_canopy_metrics, on='TC_ID', how='inner')

In [ ]:
# Convert certain coloumns to float

columns_to_convert = ['TC_P_P', 'TC_E_P']
for columns in columns_to_convert:
   studyarea_lc[columns] = studyarea_lc[columns].astype(float)

In [ ]:
# Train pipeline for image classification

train = gpd.read_file("./data/training.shp")

pipe = Pipeline([('rf', RandomForestClassifier(n_estimators=100,
                                            random_state=0))])

In [ ]:
# THIS CELL TAKES A LONG TIME
# Perform classification (1 - Tree Cover, 2 - Impervious, 3 - Water, 4 - Pervious/other Vegetation)

# with gw.open("./data/m_3411852_ne_11_060_20220504.tif", chunks=1280) as src:
#     y= fit_predict(src, pipe, train, col='class')

# Save classification image to disk
# gw.save(data=y, filename='./data/image_classified.tif', overwrite=True)

In [ ]:
# Identify census blocks for each pixel

polygon = gpd.read_file("./data/StudyArea_CensusBlock.shp")
in_path = './data/image_classified.tif'

with gw.open(in_path) as src:
    df = src.gw.extract(polygon,
                        band_names=src.band.values.tolist())
    print(df)

In [ ]:
# Summarize pixel count by category per census block

df_class_1 = df[df[1] == 1.0]
df_class_2 = df[df[1] == 2.0]
df_class_3 = df[df[1] == 3.0]
df_class_4 = df[df[1] == 4.0]

stat_1 = df_class_1.groupby('GEOID10').count()
stat_1_drop = stat_1.drop(['id', 'point', 'geometry',
                           'CTBG10', 'CT10', 'AreaSqMil',
                           'LABEL', 'FIP10', 'FIP10RV',
                           'CDP_NAME', 'CITYNAME', 'COMMNAME',
                           'Shape_Leng', 'Shape_Area'], axis=1)

stat_2 = df_class_2.groupby('GEOID10').count()
stat_2_drop = stat_2.drop(['id', 'point', 'geometry',
                           'CTBG10', 'CT10', 'AreaSqMil',
                           'LABEL', 'FIP10', 'FIP10RV',
                           'CDP_NAME', 'CITYNAME', 'COMMNAME',
                           'Shape_Leng', 'Shape_Area'], axis=1)

stat_3 = df_class_3.groupby('GEOID10').count()
stat_3_drop = stat_3.drop(['id', 'point', 'geometry',
                           'CTBG10', 'CT10', 'AreaSqMil',
                           'LABEL', 'FIP10', 'FIP10RV',
                           'CDP_NAME', 'CITYNAME', 'COMMNAME',
                           'Shape_Leng', 'Shape_Area'], axis=1)

stat_4 = df_class_4.groupby('GEOID10').count()
stat_4_drop = stat_4.drop(['id', 'point', 'geometry',
                           'CTBG10', 'CT10', 'AreaSqMil',
                           'LABEL', 'FIP10', 'FIP10RV',
                           'CDP_NAME', 'CITYNAME', 'COMMNAME',
                           'Shape_Leng', 'Shape_Area'], axis=1)

In [ ]:
# Create summary table of pixel counts and 2022 percentage of tree canopy per census block

summary_table = pd.DataFrame({'GEOID10':[], 'Tree_Cover':[], 'Impervious':[], 'Water':[], 'Pervious':[]})
summary_table['GEOID10'] = polygon['GEOID10']

merged_df = pd.merge(summary_table, stat_1_drop, on='GEOID10', how='inner')
merged_df['Tree_Cover'] = merged_df[1]
df_1 = merged_df.drop(['TC_ID', 1], axis=1)

merged_df_2 = pd.merge(df_1, stat_2_drop, on='GEOID10', how='inner')
merged_df_2['Impervious'] = merged_df_2[1]
df_2 = merged_df_2.drop(['TC_ID', 1], axis=1)

merged_df_3 = pd.merge(df_2, stat_3_drop, on='GEOID10', how='inner')
merged_df_3['Water'] = merged_df_3[1]
df_3 = merged_df_3.drop(['TC_ID', 1], axis=1)

merged_df_4 = pd.merge(df_3, stat_4_drop, on='GEOID10', how='inner')
merged_df_4['Pervious'] = merged_df_4[1]
final = merged_df_4.drop([1], axis=1)

final['TC_C_P'] = (final['Tree_Cover']) / (final['Tree_Cover'] + final['Impervious'] + final['Water'] + final['Pervious'])

studyarea_final = pd.merge(studyarea_lc, final, on='GEOID10', how='inner')

In [ ]:
# Create a ranking index for each census block based on percentage of existing tree canopy in 2016, potential tree canopy as of 2016, and tree canopy percentage in 2022
# Areas with low existing and higher potential will be prioritized for more trees

def normalize(col):
    return (col - col.min()) / (col.max() - col.min())

studyarea_final['norm_estimate'] = 1 - normalize(studyarea_final['TC_E_P'])
studyarea_final['norm_potential'] = normalize(studyarea_final['TC_P_P'])
studyarea_final['norm_current'] = 1 - normalize(studyarea_final['TC_C_P'])

studyarea_final['veg_score'] = (
    0.25 * studyarea_final['norm_estimate'] +
    0.25 * studyarea_final['norm_potential'] +
    0.5 * studyarea_final['norm_current'])

In [ ]:
# Plot results of ranking, areas in yellow are highest priority for more trees

ax = studyarea_final.plot(column='veg_score', cmap='summer', figsize=(10, 10), legend=True)
ax.set_xticks([])
ax.set_yticks([])
ax.set_title("Tree Canopy Ranking by Census Block in Study Area\nHigh Rank = High Recommendation")

In [ ]:
# Identify top 5 census tracks in need of additional tree canopy in study area

final_sorted = studyarea_final.sort_values(by="veg_score", ascending=False)
top_5 = final_sorted.head(5)
final_df = pd.DataFrame()
final_df["Common Name"] = top_5['COMMNAME']
final_df["Census Block"] = top_5['CT10']
final_df["Tree Canopy Rank"] = top_5['veg_score'].round(2)
final_df